In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import pmdarima as pm
from datetime import datetime, timedelta

In [60]:
traindata = pd.read_csv('data/traindata(ch).csv')
#testdata = pd.read_csv('test_data.csv')

In [61]:
stock = pd.read_csv('output.csv')

In [62]:
traindata['date']=pd.to_datetime(traindata['date'])
stock['date']=pd.to_datetime(stock['date'])
traindata=traindata.sort_values('date')
stock=stock.sort_values('date')
traindata = traindata.reset_index(drop=True)
stock = stock.reset_index(drop=True)

In [63]:
traindata

,index,date,company,code,P/N,title,news
0,17134,2016-03-22,尚荣医疗,2551,1,尚荣医疗：与兰州新区签订战略合作协议 项目总投资20 25亿,中证网讯 （002551）3月21日晚间发布公告称，为促进兰州新区医疗卫生事业的快速发展，推...
1,17133,2016-03-22,尚荣医疗,2551,1,尚荣医疗：与兰州新区管委会签署战略合作框架协议,尚荣医疗：与兰州新区管委会签署战略合作框架协议
2,17126,2016-03-22,姚记扑克,2605,1,姚记扑克：拟以2.55亿元对价收购万盛达扑克85%股权,姚记扑克：拟以2.55亿元对价收购万盛达扑克85%股权
3,17129,2016-03-22,姚记扑克,2605,1,姚记扑克：拟与电魂创投共同设立上海跳刀公司涉足游戏研发,姚记扑克：拟与电魂创投共同设立上海跳刀公司涉足游戏研发
4,6848,2016-03-24,富瑞特装,300228,0,富瑞特装董事长减持350万股,中国证券网讯（记者 陈天弋）公告，公司董事长邬品芳于3月23日通过深圳证券交易所大宗交易系统...
5,17071,2016-03-24,龙元建设,600491,1,龙元建设中标宁波“三路一桥”PPP项目,中证网讯 （600491） 3月23日晚间发布公告，公司近日收到宁波市住房和城乡建设委员会签...
6,6854,2016-03-24,龙韵股份,603729,0,龙韵股份股东上海领锐拟减持至多250万股,中国证券网讯（记者 严政）龙韵股份3月23日晚间公告称，公司股东上海领锐创业投资有限公司（简...
7,6852,2016-03-24,龙韵股份,603729,0,上海领锐拟减持龙韵股份不超250万股,上海领锐拟减持龙韵股份不超250万股
8,6850,2016-03-24,富瑞特装,300228,0,富瑞特装董事长减持350万股,中证网讯 （300228）3月23日晚间公告，富瑞特装董事长邬品芳于2016年3月23日通过...
9,6846,2016-03-24,富瑞特装,300228,0,富瑞特装董事长减持350万股,富瑞特装董事长减持350万股


In [28]:
stock

,date,high,close,open,num_trades,limit_down,low,limit_up,volume,total_turnover,id
0,2016-01-04,9.4815,8.9298,9.4579,24835.0,8.5042,8.8510,10.3958,67619744.0,6.603762e+08,1
1,2016-01-04,2.4154,2.2692,2.3692,11143.0,2.1538,2.2538,2.6308,51946891.0,1.223932e+08,600022
2,2016-01-04,6.8794,6.2062,6.8289,18246.0,6.1641,6.1894,7.5358,59399243.0,4.717542e+08,600177
3,2016-01-04,12.8003,11.4367,12.6044,5603.0,11.4135,11.4187,13.9499,14261856.0,1.738127e+08,18
4,2016-01-04,24.1601,24.1601,24.1601,0.0,0.0000,24.1601,0.0000,0.0,0.000000e+00,300410
5,2016-01-04,12.5024,11.0534,12.3276,5371.0,11.0534,11.0534,13.5098,12122398.0,1.552829e+08,300041
6,2016-01-04,10.1167,8.9860,9.9779,27517.0,8.9761,8.9761,10.9796,34383941.0,3.304295e+08,2471
7,2016-01-04,19.1683,19.1683,19.1683,0.0,0.0000,19.1683,0.0000,0.0,0.000000e+00,600122
8,2016-01-04,14.6338,13.3113,14.6338,19614.0,13.1989,13.2681,16.1378,35583234.0,5.748889e+08,625
9,2016-01-04,17.9900,16.1600,17.9700,13020.0,16.1600,16.1600,19.7600,13773363.0,2.303108e+08,760


In [71]:
for i in range(len(traindata)):
    print(i)
    newsdate = traindata.loc[i, "date"]
    id = traindata.loc[i, "code"]
    stockdf = stock[stock['id'] == id]
    stockdf = stockdf[['date','close']]
    if len(stockdf) > 20: 
        stockdf = stockdf[(stockdf['date'] >= newsdate - timedelta(days=30))&(stockdf['date'] <= newsdate)]
        if (len(stockdf) > 1): 
            if len(stockdf[stockdf['date'] == newsdate])==1:
                stockdf = stockdf.set_index('date',drop=True)
                #print(stockdf)
                train = stockdf[:-1]
                test = stockdf[-1:]
                train = train['close']
                test = test['close']
                #print(train)
                #print(test)
                try:
                    model = ARIMA(train, order=(5,1,1))
                    model_fit = model.fit(disp=False)
                    forecast, stderr, conf = model_fit.forecast()
                    todayclose = test[0]
                    lastclose=train[-1]
                    timeseriesforecast=forecast
                    timeserieslower = conf[0][0]
                    timeseriesupper = conf[0][1]
                    traindata.loc[i,'todayclose'] = todayclose
                    traindata.loc[i,'lastclose'] = lastclose
                    traindata.loc[i,'timeseriesforecast'] = timeseriesforecast
                    traindata.loc[i,'timeserieslower'] = timeserieslower
                    traindata.loc[i,'timeseriesupper'] = timeseriesupper
                except:
                    todayclose = test[0]
                    lastclose=train[-1]
                    traindata.loc[i,'todayclose'] = todayclose
                    traindata.loc[i,'lastclose'] = lastclose


0
1
2
3


/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency B will be used.
  % freq, ValueWarning)
/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency B will be used.
  % freq, ValueWarning)
/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency B will be used.
  % freq, ValueWarning)
/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency B will be used.
  % freq, ValueWarning)


4
5


/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency B will be used.
  % freq, ValueWarning)
/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency B will be used.
  % freq, ValueWarning)


6


...

/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequen

7494
7495
7496
7497
7498
7499
7500
7501
7502
7503
7504
7505
7506
7507
7508
7509
7510
7511
7512
7513
7514
7515
7516
7517
7518
7519
7520
7521
7522


/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequen

7523
7524
7525
7526
7527
7528
7529
7530
7531
7532
7533
7534
7535
7536
7537
7538
7539
7540
7541
7542
7543
7544
7545
7546
7547
7548
7549


/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequen

In [ ]:
s = 10
for row in range(s):
    train = df[row:(row+30)]
    test = df[(row+30):(row+31)]
    train = train['Close']
    test = test['Close']
    model = ARIMA(train, order=(5,1,1))
    model_fit = model.fit(disp=False)
    forecast, stderr, conf = model_fit.forecast()
    print('Expected: %.3f' % test[0])
    print('Forecast: %.3f' % forecast)
    print('Standard Error: %.3f' % stderr)
    print('95%% Prediction Interval: %.3f to %.3f' % (conf[0][0], conf[0][1]))

In [38]:
stock['id'].nunique()

659

In [39]:
traindata['code'].nunique()

1706

In [74]:
traindata1 = traindata[['date','company','code','P/N','title','news','todayclose','lastclose','timeseriesforecast','timeserieslower','timeseriesupper']]


In [75]:
traindata1.to_csv('traindata0330.csv',index=False)